In [ ]:
%cd /home/parthgandhi/TradeBot

In [ ]:
import polars as pl
from src.config.market import Market
from src.config.exchange import Exchange
from src.config.storage_layout import StorageLayout
from src.config.exchange_tables import EXCHG_TABLES
from src.config.brokers.nse import NSEConfig
from src.scans.swing_scan import add_basic_indicators

In [ ]:
END_DATE = "2026-01-05"

# Price Data

In [ ]:
price_db_path = StorageLayout.db_path(
    market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)
table_id = EXCHG_TABLES[Exchange.NSE]["equity_ohlcv_daily"]
query = f"""
select *
from {table_id}
"""

price_df = pl.read_database_uri(query=query, uri=f"sqlite:///{price_db_path}")

print(f"Price Data Shape: {price_df.shape}")
print(f"Total Symbols in Price Data: {price_df.select('symbol').n_unique()}")

price_df = add_basic_indicators(data=price_df).with_columns(
    pl.col("timestamp").cast(pl.String())
)

# Classification Data

In [ ]:
ind_db_path = StorageLayout.db_path(market=Market.INDIA, exchange=Exchange.NSE)

db_path = StorageLayout.db_path(market=Market.INDIA, exchange=Exchange.NSE)

max_date_query = f"""
select max(timestamp) as timestamp
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
"""

max_date = pl.read_database_uri(query=max_date_query, uri=f"sqlite:///{db_path}").item(
    0, 0
)

print(f"MAX DATE of NSE SECTORS: {max_date}")

industry_query = f"""
select distinct *
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
where timestamp = '{max_date}'
"""

nse_classify_df = pl.read_database_uri(
    query=industry_query, uri=f"sqlite:///{db_path}"
).select(pl.exclude("timestamp"))

print(f"NSE CLASSIFICATION SHAPE: {nse_classify_df.shape}")

nse_classify_df = (
    nse_classify_df.lazy()
    .remove(pl.any_horizontal(pl.col("*").is_null()))
    .sort("market_cap_cr", descending=True)
    .with_row_index(name="rank", offset=1)
    .with_columns(
        pl.when(pl.col("rank").is_between(1, 100, closed="both"))
        .then(pl.lit("Large Cap"))
        .otherwise(
            pl.when(pl.col("rank").is_between(101, 250, closed="both"))
            .then(pl.lit("Mid Cap"))
            .otherwise(pl.lit("Small Cap"))
        )
        .alias("category")
    )
    .collect()
)


print(f"NSE CLASSIFICATION SHAPE AFTER NULL MARKET CAP FILTER: {nse_classify_df.shape}")

# Combine 

In [ ]:
final_df = (
    price_df.join(nse_classify_df.lazy(), on="symbol")
    .remove(pl.any_horizontal(pl.col("*").is_null()))
    .collect()
)

In [ ]:
final_df.lazy().group_by(
    "timestamp",
).agg(
    [pl.col("category").count().alias("total_stocks")]
    + [
        (pl.col("close") >= pl.col(f"close_ema_{i}"))
        .sum()
        .alias(f"stocks_above_ema_{i}")
        for i in [9, 21]
    ]
    + [
        (pl.col("close") >= pl.col(f"close_sma_{i}"))
        .sum()
        .alias(f"stocks_above_sma_{i}")
        for i in [50, 200]
    ]
).with_columns(
    (pl.col(col) * 100 / pl.col("total_stocks")).round(2).alias(f"{col}_pct")
    for col in [
        "stocks_above_ema_9",
        "stocks_above_ema_21",
        "stocks_above_sma_50",
        "stocks_above_sma_200",
    ]
).select(
    pl.exclude(
        [
            "total_stocks",
            "stocks_above_ema_9",
            "stocks_above_ema_21",
            "stocks_above_sma_50",
            "stocks_above_sma_200",
        ]
    )
).sort("timestamp", descending=True).head(10).collect()